# ECDLP
Elliptic Curve Discrete Logarithm Problem

Let:
- 𝐸 be an elliptic curve over a finite field Fp
- 𝑃 be a publicly known point on the curve
- 𝑄 = 𝑘𝑃 be another publicly known point, where 𝑘 is a secret integer (scalar multiplication on the elliptic curve)

The ECDLP is:
Given points 𝑃 and 𝑄 = 𝑘𝑃, find the integer 𝑘

## Helper Functions

In [51]:
# Imports and Constants
import random
import math
import time
import statistics
from functools import wraps

INF = None  # point at infinity

In [52]:
# timing helper

def time_call(fn, *args, **kwargs):
    """Return (result, elapsed_seconds) for a single function call."""
    start = time.perf_counter()
    result = fn(*args, **kwargs)
    end = time.perf_counter()
    return result, end - start

In [53]:
# random generator
import random

def random_k(n):
    return random.randrange(1, n)

In [54]:
# number theory operations
def egcd(a, b):
    """Extended Euclidean algorithm: returns (g, x, y) s.t. ax + by = g = gcd(a, b)."""
    if b == 0:
        return (a, 1, 0)
    g, x1, y1 = egcd(b, a % b)
    x = y1
    y = x1 - (a // b) * y1
    return (g, x, y)


def inv_mod_order(a, n):
    """
    Modular inverse of a modulo n, where n can be composite.
    Returns None if gcd(a, n) != 1 (i.e. no inverse).
    """
    a %= n
    g, x, _ = egcd(a, n)
    if g != 1:
        return None
    return x % n

def crt_pair(a1, m1, a2, m2):
    """Combine x ≡ a1 (mod m1), x ≡ a2 (mod m2) into x (mod m1*m2)."""
    # solve a1 + t*m1 ≡ a2 (mod m2)
    # t*m1 ≡ (a2 - a1) (mod m2)
    rhs = (a2 - a1) % m2
    inv_m1 = inv_mod_order(m1, m2)
    if inv_m1 is None:
        raise ValueError("m1 not invertible modulo m2 in CRT.")
    t = (rhs * inv_m1) % m2
    x = a1 + t * m1
    return x % (m1 * m2), m1 * m2

## Elliptic Curve Implementation

In [ ]:
class EllipticCurveGroup:
    """
    Elliptic curve group over a prime field F_p, y^2 = x^3 + a x + b (mod p),
    with base point G and (optionally) known order n and factorisation of n.

    Works for prime or composite n.
    """

    def __init__(self, p, a, b, G, n=None, factor_n=None, name=None):
        self.p = int(p)
        self.a = int(a)
        self.b = int(b)
        self.G = G  # tuple (x, y) or None
        self.n = int(n) if n is not None else None
        self.factor_n = [(int(q), int(e)) for (q, e) in factor_n] if factor_n is not None else None
        self.name = name or "ECGroup"

    # ----- field & curve ops -----

    def field_inv(self, x):
        """Inverse in F_p, p is prime."""
        return pow(x, self.p - 2, self.p)

    def is_on_curve(self, P):
        if P is INF:
            return True
        x, y = P
        lhs = (y * y) % self.p
        rhs = (x * x * x + self.a * x + self.b) % self.p
        return lhs == rhs

    def add(self, P, Q):
        """Point addition P + Q on this curve."""
        if P is INF:
            return Q
        if Q is INF:
            return P

        x1, y1 = P
        x2, y2 = Q

        # P + (-P) = INF
        if x1 == x2 and (y1 + y2) % self.p == 0:
            return INF

        if P != Q:
            # slope = (y2 - y1) / (x2 - x1)
            m = (y2 - y1) * self.field_inv((x2 - x1) % self.p) % self.p
        else:
            # slope = (3 x1^2 + a) / (2 y1)
            m = (3 * x1 * x1 + self.a) * self.field_inv((2 * y1) % self.p) % self.p

        x3 = (m * m - x1 - x2) % self.p
        y3 = (m * (x1 - x3) - y1) % self.p
        return (x3, y3)

    def scalar_mul(self, k, P=None):
        """Double-and-add scalar multiplication k*P (or k*G if P is None)."""
        if P is None:
            P = self.G
        k = int(k)
        R = INF
        Q = P
        while k > 0:
            if k & 1:
                R = self.add(R, Q)
            Q = self.add(Q, Q)
            k >>= 1
        return R

    def random_scalar(self):
        if self.n is None:
            raise ValueError("Group order n is unknown; cannot pick random scalar in [1, n-1].")
        return random.randrange(1, self.n)
    
    # ----- Pollard's rho for ECDLP -----

    @staticmethod
    def _partition(P):
        """Simple 3-way partition based on x-coordinate."""
        if P is INF:
            return 0
        x, _ = P
        return x % 3

    def _rho_step(self, X, a_coeff, b_coeff, P_target):
        i = self._partition(X)
        n = self.n
        if n is None:
            raise ValueError("Need group order n to run Pollard's rho.")

        if i == 0:
            # X = X + G
            X_new = self.add(X, self.G)
            a_new = (a_coeff + 1) % n
            b_new = b_coeff
        elif i == 1:
            # X = 2X
            X_new = self.add(X, X)
            a_new = (2 * a_coeff) % n
            b_new = (2 * b_coeff) % n
        else:
            # X = X + P_target
            X_new = self.add(X, P_target)
            a_new = a_coeff
            b_new = (b_coeff + 1) % n
        return X_new, a_new, b_new

    def pollards_rho(self, P_target, max_iters=10_000_000, log_every=None, verbose=True):
        """
        Pollard's rho for ECDLP: find k s.t. P_target = k * G.

        Returns (k or None, iterations_used).
        """
        if self.G is None or self.n is None:
            raise ValueError("Need base point G and order n to run Pollard's rho.")

        n = self.n

        # randomize start
        a1 = random.randrange(0, n)
        b1 = random.randrange(0, n)
        X = self.add(self.scalar_mul(a1, self.G), self.scalar_mul(b1, P_target))

        a2, b2 = a1, b1
        X2 = X

        for it in range(1, max_iters + 1):
            # 1 step (tortoise)
            X,  a1, b1  = self._rho_step(X,  a1, b1,  P_target)
            # 2 steps (hare)
            X2, a2, b2  = self._rho_step(X2, a2, b2, P_target)
            X2, a2, b2  = self._rho_step(X2, a2, b2, P_target)

            if verbose and log_every is not None and it % log_every == 0:
                print(f"[{self.name} rho] iter={it:,}")

            if X == X2:
                if verbose:
                    print(f"[{self.name} rho] collision after {it:,} iterations")

                num = (a1 - a2) % n
                den = (b2 - b1) % n

                g = math.gcd(den, n)
                if g == 0 or g == n:
                    if verbose:
                        print(f"[{self.name} rho] degenerate collision (g={g}); restart run")
                    return None, it

                if g != 1:
                    if verbose:
                        print(f"[{self.name} rho] nontrivial gcd(g={g}) with n={n} (factor of group order)")
                        print("  Treating this run as failure; you could use g to factor n.")
                    return None, it

                inv_den = inv_mod_order(den, n)
                if inv_den is None:
                    if verbose:
                        print(f"[{self.name} rho] den not invertible even though gcd=1? (unexpected)")
                    return None, it

                k = (num * inv_den) % n
                return k, it

        if verbose:
            print(f"[{self.name} rho] reached max_iters={max_iters:,} without useful collision")
        return None, max_iters
    
    # ----- Pohlig-Hellman (composite order) -----

    def _dlog_prime_power(self, q, e, P_target, verbose=True):
        """
        Solve k such that P_target = k * G in the subgroup of order q^e.

        Uses straightforward brute-force over q^e 
        
        Returns k modulo q^e.
        """
        if self.n is None:
            raise ValueError("Need group order n for Pohlig-Hellman.")

        n = self.n
        q_e = q ** e
        h = n // q_e

        # Generator of subgroup of order q^e
        Gqe = self.scalar_mul(h, self.G)
        Pqe = self.scalar_mul(h, P_target)

        R = INF
        for k in range(q_e):
            if R == Pqe:
                if verbose:
                    print(f"    [PP] solved k ≡ {k} (mod {q_e}) in {k+1} steps")
                return k
            R = self.add(R, Gqe)

        if verbose:
            print(f"    [PP] FAILED mod {q}^{e}")
        return None

    def pohlig_hellman(self, P_target, verbose=True):
        if self.n is None or self.factor_n is None:
            raise ValueError("Need n and factor_n for Pohlig-Hellman.")

        n = self.n
        fac = self.factor_n

        if verbose:
            print(f"[{self.name} PH] starting Pohlig-Hellman")
            print("  factors:", fac)

        congruences = []
        for (q, e) in fac:
            if verbose:
                print(f"  [PH] solving modulo q^e = {q}^{e}")

            k_mod_qe = self._dlog_prime_power(q, e, P_target, verbose=verbose)
            congruences.append((k_mod_qe, q ** e))

            if verbose:
                print(f"    ⇒ congruence: k ≡ {k_mod_qe} (mod {q ** e})")

        # Combine via CRT
        k, m = congruences[0]
        for (ai, mi) in congruences[1:]:
            prev_k, prev_m = k, m
            k, m = crt_pair(k, m, ai, mi)
            if verbose:
                print(f"    [CRT] combining: k ≡ {prev_k} (mod {prev_m})  and  k ≡ {ai} (mod {mi})")
                print(f"    [CRT]   ⇒ new solution: k ≡ {k} (mod {m})")

        k_final = k % n
        if verbose:
            print(f"[{self.name} PH] final solution k ≡ {k_final} (mod {n})")

        return k_final
    
    # ----- subgroup helpers -----

    def largest_prime_factor(self):
        """
        Return the largest prime factor of n, assuming factor_n is set.
        """
        if self.factor_n is None:
            raise ValueError("factor_n is not set; cannot determine largest prime factor.")
        return max(q for (q, e) in self.factor_n)

    def subgroup_generator(self, q):
        """
        Given a prime factor q of n, return a generator of the subgroup of order q:
            G_q = (n / q) * G

        Also sanity-check that q * G_q = INF.
        """
        if self.n is None:
            raise ValueError("Group order n is not set.")
        if self.n % q != 0:
            raise ValueError(f"{q} does not divide group order n={self.n}.")

        h = self.n // q
        G_q = self.scalar_mul(h, self.G)

        # sanity: check order divides q
        P = self.scalar_mul(q, G_q)
        if P is not INF:
            raise ValueError("Constructed G_q does not have order dividing q. Something's off.")

        return G_q
    
    def pollards_rho_subgroup(self, G_sub, P_target, q,
                              max_iters=10_000_000, log_every=None, verbose=True):
        """
        Pollard's rho to solve k such that P_target = k * G_sub
        in the *prime-order* subgroup of size q.

        Uses the same partition function & group law, but the modulus is q (not full n).
        Returns (k or None, iterations_used).
        """
        q = int(q)

        # randomize start
        a1 = random.randrange(0, q)
        b1 = random.randrange(0, q)
        X = self.add(self.scalar_mul(a1, G_sub), self.scalar_mul(b1, P_target))

        a2, b2 = a1, b1
        X2 = X

        for it in range(1, max_iters + 1):
            # 1 step (tortoise)
            X,  a1, b1  = self._rho_step(X,  a1, b1,  P_target)
            # 2 steps (hare)
            X2, a2, b2  = self._rho_step(X2, a2, b2, P_target)
            X2, a2, b2  = self._rho_step(X2, a2, b2, P_target)

            if verbose and log_every is not None and it % log_every == 0:
                print(f"[{self.name} rho-subgroup] iter={it:,}")

            if X == X2:
                if verbose:
                    print(f"[{self.name} rho-subgroup] collision after {it:,} iterations")

                num = (a1 - a2) % q
                den = (b2 - b1) % q

                g = math.gcd(den, q)
                if g == 0 or g == q:
                    if verbose:
                        print(f"[{self.name} rho-subgroup] degenerate collision (g={g}); restart run")
                    return None, it

                if g != 1:
                    # q is prime, so gcd != 1 implies den ≡ 0; treat as failure
                    if verbose:
                        print(f"[{self.name} rho-subgroup] unexpected gcd(g={g}) with prime q={q}")
                    return None, it

                inv_den = inv_mod_order(den, q)
                if inv_den is None:
                    if verbose:
                        print(f"[{self.name} rho-subgroup] den not invertible modulo q (unexpected)")
                    return None, it

                k = (num * inv_den) % q
                return k, it

        if verbose:
            print(f"[{self.name} rho-subgroup] reached max_iters={max_iters:,} without useful collision")
        return None, max_iters

## Prime order curve

In [159]:
# Parameters from Sage for PRIME-ORDER curve
p  = 809251356833
a  = 728390363442
b  = 371490390854
Gx = 407440511262
Gy = 76513945770
n  = 809251644361

G = (Gx, Gy)

prime_curve = EllipticCurveGroup(
    p=p,
    a=a,
    b=b,
    G=G,
    n=n,
    factor_n=None,   # prime order
    name="PrimeCurve"
)

# sanity check
assert prime_curve.is_on_curve(G)
assert prime_curve.scalar_mul(n, G) is INF

# Print curve size and largest factor size
curve_bits = n.bit_length()   # or use p_c.bit_length()
largest_factor = n
largest_factor_bits = largest_factor.bit_length()

print(f"[{prime_curve.name}] Group order n is {n.bit_length()} bits")
print(f"[{prime_curve.name}] Largest prime factor = {largest_factor} ({largest_factor_bits} bits)")

[PrimeCurve] Group order n is 40 bits
[PrimeCurve] Largest prime factor = 809251644361 (40 bits)


In [161]:
# Pollard's rho on prime-order curve
k_secret = prime_curve.random_scalar()
P_target = prime_curve.scalar_mul(k_secret)

(k_found, iters), t = time_call(
    prime_curve.pollards_rho,
    P_target,
    10_000_000,
    100_000,          # log_every
    True          # verbose
)

print("Prime curve - secret k:", k_secret)
print("Prime curve - found  k:", k_found)
print("success:", k_found == k_secret)
print("iters:", iters, "time:", t, "s")

[PrimeCurve rho] iter=100,000
[PrimeCurve rho] iter=200,000
[PrimeCurve rho] iter=300,000
[PrimeCurve rho] iter=400,000
[PrimeCurve rho] iter=500,000
[PrimeCurve rho] iter=600,000
[PrimeCurve rho] iter=700,000
[PrimeCurve rho] iter=800,000
[PrimeCurve rho] iter=900,000
[PrimeCurve rho] iter=1,000,000
[PrimeCurve rho] iter=1,100,000
[PrimeCurve rho] iter=1,200,000
[PrimeCurve rho] collision after 1,279,182 iterations
Prime curve - secret k: 471532524194
Prime curve - found  k: 471532524194
success: True
iters: 1279182 time: 18.499352300001192 s


## Composite order curve


In [149]:
# Parameters from Sage for COMPOSITE-ORDER curve
p_c  = 1035972675893
a_c  = 206712200754
b_c  = 139762077084
Gc_x = 81155312250
Gc_y = 447657618032
n_c  = 1035972485265
factor_n_c = [(3, 1), (5, 1), (1613, 1), (5651, 1), (7577, 1)]

Gc = (Gc_x, Gc_y)

weak_curve = EllipticCurveGroup(
    p=p_c,
    a=a_c,
    b=b_c,
    G=Gc,
    n=n_c,
    factor_n=factor_n_c,
    name="CompositeCurve"
)

# sanity check
assert weak_curve.is_on_curve(Gc)
assert weak_curve.scalar_mul(n_c, Gc) is INF

# Print curve size and largest factor size
curve_bits = n_c.bit_length()   # or use p_c.bit_length()
largest_factor = max(q for (q, e) in factor_n_c)
largest_factor_bits = largest_factor.bit_length()

print(f"[{weak_curve.name}] Group order n is {n_c.bit_length()} bits")
print(f"[{weak_curve.name}] Largest prime factor = {largest_factor} ({largest_factor_bits} bits)")


[CompositeCurve] Group order n is 40 bits
[CompositeCurve] Largest prime factor = 7577 (13 bits)


In [150]:
# Pollard's rho on composite-order curve
k_secret_c = weak_curve.random_scalar()
P_target_c = weak_curve.scalar_mul(k_secret_c)

(k_found_c, iters_c), t_c = time_call(
    weak_curve.pollards_rho,
    P_target_c,
    10_000_000,
    100_000,
    True
)

print("Composite curve - secret k:", k_secret_c)
print("Composite curve - found  k:", k_found_c)
print("success:", k_found_c == k_secret_c)
print("iters:", iters_c, "time:", t_c, "s")

[CompositeCurve rho] iter=100,000
[CompositeCurve rho] iter=200,000
[CompositeCurve rho] iter=300,000
[CompositeCurve rho] iter=400,000
[CompositeCurve rho] iter=500,000
[CompositeCurve rho] iter=600,000
[CompositeCurve rho] iter=700,000
[CompositeCurve rho] iter=800,000
[CompositeCurve rho] iter=900,000
[CompositeCurve rho] iter=1,000,000
[CompositeCurve rho] iter=1,100,000
[CompositeCurve rho] iter=1,200,000
[CompositeCurve rho] iter=1,300,000
[CompositeCurve rho] iter=1,400,000
[CompositeCurve rho] iter=1,500,000
[CompositeCurve rho] iter=1,600,000
[CompositeCurve rho] collision after 1,601,331 iterations
[CompositeCurve rho] nontrivial gcd(g=3) with n=1035972485265 (factor of group order)
  Treating this run as failure; you could use g to factor n.
Composite curve - secret k: 1000244843534
Composite curve - found  k: None
success: False
iters: 1601331 time: 23.19821519999823 s


In [152]:
# Pohlig-Hellman on composite-order curve
k_secret_ph = weak_curve.random_scalar()
P_target_ph = weak_curve.scalar_mul(k_secret_ph)

k_ph, t_ph = time_call(
    weak_curve.pohlig_hellman,
    P_target_ph,
    verbose=True
)

print("PH - secret k:", k_secret_ph)
print("PH - found  k:", k_ph)
print("success:", (k_ph % n_c) == (k_secret_ph % n_c))
print("PH time:", t_ph, "s")

[CompositeCurve PH] starting Pohlig-Hellman
  factors: [(3, 1), (5, 1), (1613, 1), (5651, 1), (7577, 1)]
  [PH] solving modulo q^e = 3^1
    [PP] solved k ≡ 2 (mod 3) in 3 steps
    ⇒ congruence: k ≡ 2 (mod 3)
  [PH] solving modulo q^e = 5^1
    [PP] solved k ≡ 1 (mod 5) in 2 steps
    ⇒ congruence: k ≡ 1 (mod 5)
  [PH] solving modulo q^e = 1613^1
    [PP] solved k ≡ 203 (mod 1613) in 204 steps
    ⇒ congruence: k ≡ 203 (mod 1613)
  [PH] solving modulo q^e = 5651^1
    [PP] solved k ≡ 1263 (mod 5651) in 1264 steps
    ⇒ congruence: k ≡ 1263 (mod 5651)
  [PH] solving modulo q^e = 7577^1
    [PP] solved k ≡ 5384 (mod 7577) in 5385 steps
    ⇒ congruence: k ≡ 5384 (mod 7577)
    [CRT] combining: k ≡ 2 (mod 3)  and  k ≡ 1 (mod 5)
    [CRT]   ⇒ new solution: k ≡ 11 (mod 15)
    [CRT] combining: k ≡ 11 (mod 15)  and  k ≡ 203 (mod 1613)
    [CRT]   ⇒ new solution: k ≡ 9881 (mod 24195)
    [CRT] combining: k ≡ 9881 (mod 24195)  and  k ≡ 1263 (mod 5651)
    [CRT]   ⇒ new solution: k ≡ 70175381 

### Benchmarks


In [61]:
def benchmark_pollard(curve: EllipticCurveGroup, num_trials=5, verbose=False):
    """
    Run Pollard's rho num_trials times on a given curve.
    Returns list of result dicts.
    """
    results = []
    print(f"=== Pollard’s rho on {curve.name} (|G| ≈ {curve.n.bit_length()} bits) ===")

    for i in range(num_trials):
        k_secret = curve.random_scalar()
        P_target = curve.scalar_mul(k_secret)

        # curve.pollards_rho returns (k, iters)
        (k_found, iters), t = time_call(
            curve.pollards_rho,
            P_target,
            10_000_000,   # max_iters
            None,         # log_every
            verbose       # verbose flag
        )

        success = (k_found == k_secret)
        print(f"trial {i+1}: time={t:.4f} s, success={success}")
        results.append({
            "trial": i + 1,
            "k_secret": k_secret,
            "k_found": k_found,
            "success": success,
            "time": t,
            "iters": iters,
        })

    # summary
    times = [r["time"] for r in results if r["success"]]
    avg_time = statistics.mean(times) if times else float("nan")
    print(f"Average time (successful runs): {avg_time:.4f} s over {len(times)} runs\n")

    return results

def benchmark_pohlig(weak_curve: EllipticCurveGroup, num_trials=5, verbose=False):
    """
    Run Pohlig–Hellman num_trials times on weak_curve (must have factor_n set).
    Returns list of result dicts.
    """
    if weak_curve.factor_n is None:
        raise ValueError("weak_curve.factor_n is None, cannot run Pohlig–Hellman.")

    results = []
    print(f"=== Pohlig–Hellman on {weak_curve.name} (n factors = {weak_curve.factor_n}) ===")

    for i in range(num_trials):
        k_secret = weak_curve.random_scalar()
        P_target = weak_curve.scalar_mul(k_secret)

        k_ph, t = time_call(
            weak_curve.pohlig_hellman,
            P_target,
            verbose
        )

        success = (k_ph % weak_curve.n) == (k_secret % weak_curve.n)
        print(f"trial {i+1}: time={t:.6f} s, success={success}")
        results.append({
            "trial": i + 1,
            "k_secret": k_secret,
            "k_found": k_ph,
            "success": success,
            "time": t,
        })

    times = [r["time"] for r in results if r["success"]]
    avg_time = statistics.mean(times) if times else float("nan")
    print(f"Average PH time (successful runs): {avg_time:.6f} s over {len(times)} runs\n")

    return results

In [153]:
def run_all_benchmarks(num_trials=5, verbose=False):
    """
    Run:
      - Pollard on prime_curve
      - Pollard on weak_curve (composite order)
      - Pohlig-Hellman on weak_curve

    Returns dict of all results.
    """
    print(f"\n=== Running benchmarks with {num_trials} trials each ===\n")

    prime_pollard_results = benchmark_pollard(prime_curve, num_trials=num_trials, verbose=verbose)
    #weak_pollard_results  = benchmark_pollard(weak_curve,  num_trials=num_trials, verbose=verbose)
    ph_results            = benchmark_pohlig(weak_curve,   num_trials=num_trials, verbose=verbose)

    return {
        "prime_pollard": prime_pollard_results,
        #"weak_pollard": weak_pollard_results,
        "pohlig": ph_results,
    }

In [154]:
results = run_all_benchmarks(num_trials=5, verbose=False)


=== Running benchmarks with 5 trials each ===

=== Pollard’s rho on PrimeCurve (|G| ≈ 40 bits) ===
trial 1: time=14.1814 s, success=True
trial 2: time=16.7373 s, success=True
trial 3: time=3.7237 s, success=True
trial 4: time=12.5290 s, success=True
trial 5: time=19.2340 s, success=True
Average time (successful runs): 13.2811 s over 5 runs

=== Pohlig–Hellman on CompositeCurve (n factors = [(3, 1), (5, 1), (1613, 1), (5651, 1), (7577, 1)]) ===
trial 1: time=0.029698 s, success=True
trial 2: time=0.024763 s, success=True
trial 3: time=0.034849 s, success=True
trial 4: time=0.015899 s, success=True
trial 5: time=0.050479 s, success=True
Average PH time (successful runs): 0.031137 s over 5 runs



## 128-bit example

In [87]:
def attack_largest_subgroup_with_pollard(curve: EllipticCurveGroup,
                                         max_iters=10_000_000,
                                         verbose=True):
    """
    1. Find largest prime factor q of n.
    2. Compute subgroup generator G_sub of order q.
    3. Sample random k in [1, q-1] and target P = k*G_sub.
    4. Run Pollard's rho restricted to the subgroup to recover k.
    """
    q = curve.largest_prime_factor()
    if verbose:
        print(f"[{curve.name}] largest prime factor of n is q={q} ({q.bit_length()} bits)")

    G_sub = curve.subgroup_generator(q)
    if verbose:
        print(f"[{curve.name}] constructed subgroup generator G_sub")

    k_secret = random.randrange(1, q)
    P_target = curve.scalar_mul(k_secret, G_sub)

    if verbose:
        print(f"[{curve.name}] attacking subgroup: P_target = k * G_sub with k={k_secret}")

    (k_found, iters), t = time_call(
        curve.pollards_rho_subgroup,
        G_sub, P_target, q,
        max_iters,
        100_000,     # log_every
        verbose
    )

    success = (k_found == k_secret)
    print(f"[{curve.name}] subgroup attack done.")
    print(f"  secret k = {k_secret}")
    print(f"  found  k = {k_found}")
    print(f"  success = {success}")
    print(f"  iterations = {iters:,}, time = {t:.4f} s")

    return {
        "q": q,
        "q_bits": q.bit_length(),
        "k_secret": k_secret,
        "k_found": k_found,
        "success": success,
        "iters": iters,
        "time": t,
    }

In [140]:
# Parameters from Sage for COMPOSITE-ORDER curve
p_c  = 311116767020924054633530003497992666791
a_c  = 101336601930719595799392832006028414076
b_c  = 213025625074471546599358397361042807007
Gc_x = 2169322230080032791616248567849170652
Gc_y = 282157040398817792104497502041790378950
n_c  = 155558383510462027319364776939301249336
factor_n_c = [(2, 3), (3, 1), (263, 1), (220399, 1), (269879, 1), (18546707, 1), (106486547, 1), (209790667, 1)]

Gc = (Gc_x, Gc_y)

weak_128_curve = EllipticCurveGroup(
    p=p_c,
    a=a_c,
    b=b_c,
    G=Gc,
    n=n_c,
    factor_n=factor_n_c,
    name="Composite128Curve"
)

# sanity check
assert weak_128_curve.is_on_curve(Gc)
assert weak_128_curve.scalar_mul(n_c, Gc) is INF

# Print curve size and largest factor size
curve_bits = n_c.bit_length()   # or use p_c.bit_length()
largest_factor = max(q for (q, e) in factor_n_c)
largest_factor_bits = largest_factor.bit_length()

print(f"[{weak_128_curve.name}] Group order n is {n_c.bit_length()} bits")
print(f"[{weak_128_curve.name}] Largest prime factor = {largest_factor} ({largest_factor_bits} bits)")

[Composite128Curve] Group order n is 127 bits
[Composite128Curve] Largest prime factor = 209790667 (28 bits)


In [141]:
# Pohlig-Hellman on composite-order curve
k_secret_ph = weak_128_curve.random_scalar()
P_target_ph = weak_128_curve.scalar_mul(k_secret_ph)

k_ph, t_ph = time_call(
    weak_128_curve.pohlig_hellman,
    P_target_ph,
    verbose=True
)

print("PH - secret k:", k_secret_ph)
print("PH - found  k:", k_ph)
print("success:", (k_ph % n_c) == (k_secret_ph % n_c))
print("PH time:", t_ph, "s")

[Composite128Curve PH] starting Pohlig-Hellman
  factors: [(2, 3), (3, 1), (263, 1), (220399, 1), (269879, 1), (18546707, 1), (106486547, 1), (209790667, 1)]
  [PH] solving modulo q^e = 2^3
    [PP] solved k ≡ 0 (mod 8) in 1 steps
    ⇒ congruence: k ≡ 0 (mod 8)
  [PH] solving modulo q^e = 3^1
    [PP] solved k ≡ 2 (mod 3) in 3 steps
    ⇒ congruence: k ≡ 2 (mod 3)
  [PH] solving modulo q^e = 263^1
    [PP] solved k ≡ 17 (mod 263) in 18 steps
    ⇒ congruence: k ≡ 17 (mod 263)
  [PH] solving modulo q^e = 220399^1
    [PP] solved k ≡ 65348 (mod 220399) in 65349 steps
    ⇒ congruence: k ≡ 65348 (mod 220399)
  [PH] solving modulo q^e = 269879^1
    [PP] solved k ≡ 182335 (mod 269879) in 182336 steps
    ⇒ congruence: k ≡ 182335 (mod 269879)
  [PH] solving modulo q^e = 18546707^1
    [PP] solved k ≡ 12038827 (mod 18546707) in 12038828 steps
    ⇒ congruence: k ≡ 12038827 (mod 18546707)
  [PH] solving modulo q^e = 106486547^1
    [PP] solved k ≡ 17016791 (mod 106486547) in 17016792 steps
 

In [102]:
# Parameters from Sage for COMPOSITE-ORDER curve
p_c  = 272370181271426527685337470280740111579
a_c  = 39435050263767734438343031137820564931
b_c  = 142137769887748982119023024061917804120
Gc_x = 264084654360059098080530587325593281170
Gc_y = 127701244832168292174084517150957147703
n_c  = 136185090635713263838004951342990293139
factor_n_c = [(7, 1), (17, 1), (460843, 1), (520633, 1), (1005373, 1), (1064507, 1), (1077691, 1), (4135499, 1)]

Gc = (Gc_x, Gc_y)

weak_128_curve_2 = EllipticCurveGroup(
    p=p_c,
    a=a_c,
    b=b_c,
    G=Gc,
    n=n_c,
    factor_n=factor_n_c,
    name="Composite128Curve2"
)

# sanity check
assert weak_128_curve_2.is_on_curve(Gc)
assert weak_128_curve_2.scalar_mul(n_c, Gc) is INF

# Print curve size and largest factor size
curve_bits = n_c.bit_length()   # or use p_c.bit_length()
largest_factor = max(q for (q, e) in factor_n_c)
largest_factor_bits = largest_factor.bit_length()

print(f"[{weak_128_curve_2.name}] Group order n is {n_c.bit_length()} bits")
print(f"[{weak_128_curve_2.name}] Largest prime factor = {largest_factor} ({largest_factor_bits} bits)")


[Composite128Curve2] Group order n is 127 bits
[Composite128Curve2] Largest prime factor = 4135499 (22 bits)


In [107]:
# Pohlig-Hellman on composite-order curve
k_secret_ph = weak_128_curve_2.random_scalar()
P_target_ph = weak_128_curve_2.scalar_mul(k_secret_ph)

k_ph, t_ph = time_call(
    weak_128_curve_2.pohlig_hellman,
    P_target_ph,
    verbose=True
)

print("PH - secret k:", k_secret_ph)
print("PH - found  k:", k_ph)
print("success:", (k_ph % n_c) == (k_secret_ph % n_c))
print("PH time:", t_ph, "s")

[Composite128Curve2 PH] starting Pohlig-Hellman
  factors: [(7, 1), (17, 1), (460843, 1), (520633, 1), (1005373, 1), (1064507, 1), (1077691, 1), (4135499, 1)]
  [PH] solving modulo q=7 (e=1)
  [PH] solved mod 7 in 2 steps
    ⇒ congruence: k ≡ 1 (mod 7)
  [PH] solving modulo q=17 (e=1)
  [PH] solved mod 17 in 10 steps
    ⇒ congruence: k ≡ 9 (mod 17)
  [PH] solving modulo q=460843 (e=1)
  [PH] solved mod 460843 in 433386 steps
    ⇒ congruence: k ≡ 433385 (mod 460843)
  [PH] solving modulo q=520633 (e=1)
  [PH] solved mod 520633 in 453644 steps
    ⇒ congruence: k ≡ 453643 (mod 520633)
  [PH] solving modulo q=1005373 (e=1)
  [PH] solved mod 1005373 in 915996 steps
    ⇒ congruence: k ≡ 915995 (mod 1005373)
  [PH] solving modulo q=1064507 (e=1)
  [PH] solved mod 1064507 in 977919 steps
    ⇒ congruence: k ≡ 977918 (mod 1064507)
  [PH] solving modulo q=1077691 (e=1)
  [PH] solved mod 1077691 in 293044 steps
    ⇒ congruence: k ≡ 293043 (mod 1077691)
  [PH] solving modulo q=4135499 (e=1)


In [88]:
subgroup_result = attack_largest_subgroup_with_pollard(weak_128_curve_2, max_iters=20_000_000, verbose=True)

[Composite128Curve2] largest prime factor of n is q=29326951 (25 bits)
[Composite128Curve2] constructed subgroup generator G_sub
[Composite128Curve2] attacking subgroup: P_target = k * G_sub with k=24395498
[Composite128Curve2 rho-subgroup] iter=100,000
[Composite128Curve2 rho-subgroup] iter=200,000
[Composite128Curve2 rho-subgroup] iter=300,000
[Composite128Curve2 rho-subgroup] iter=400,000
[Composite128Curve2 rho-subgroup] iter=500,000
[Composite128Curve2 rho-subgroup] iter=600,000
[Composite128Curve2 rho-subgroup] iter=700,000
[Composite128Curve2 rho-subgroup] iter=800,000
[Composite128Curve2 rho-subgroup] iter=900,000
[Composite128Curve2 rho-subgroup] iter=1,000,000
[Composite128Curve2 rho-subgroup] iter=1,100,000
[Composite128Curve2 rho-subgroup] iter=1,200,000
[Composite128Curve2 rho-subgroup] iter=1,300,000
[Composite128Curve2 rho-subgroup] iter=1,400,000
[Composite128Curve2 rho-subgroup] iter=1,500,000
[Composite128Curve2 rho-subgroup] iter=1,600,000
[Composite128Curve2 rho-sub

## 256-bit example

In [155]:
# Parameters from Sage for COMPOSITE-ORDER curve
p_c  = 89933517633481385207637133214036986443581631501882681286795367663455666098427
a_c  = 8117034959394480639773107230804084816978337927558537213638207812660328329870
b_c  = 2162853857178468415882029332256206237233013833401037903485875212143833580217
Gc_x = 4696721022079968011921854916853308985494998531006590882442542572157282203182
Gc_y = 41849223911901852158684209011186422433672455422317563768258244154369867598099
n_c  = 22483379408370346301909283303509246610875952506554597057237379567226161769924
factor_n_c = [(2, 2), (3, 1), (83, 1), (2346846930209, 1), (104741868930961, 1), (915369132816071, 1), (3278160392223083, 1), (30603472356698317, 1)]

Gc = (Gc_x, Gc_y)

weak_256_curve = EllipticCurveGroup(
    p=p_c,
    a=a_c,
    b=b_c,
    G=Gc,
    n=n_c,
    factor_n=factor_n_c,
    name="Composite256Curve"
)

# sanity check
assert weak_256_curve.is_on_curve(Gc)
assert weak_256_curve.scalar_mul(n_c, Gc) is INF

curve_bits = n_c.bit_length()   # or use p_c.bit_length()
largest_factor = max(q for (q, e) in factor_n_c)
largest_factor_bits = largest_factor.bit_length()

print(f"[{weak_256_curve.name}] Group order n is {n_c.bit_length()} bits")
print(f"[{weak_256_curve.name}] Largest prime factor = {largest_factor} ({largest_factor_bits} bits)")

[Composite256Curve] Group order n is 254 bits
[Composite256Curve] Largest prime factor = 30603472356698317 (55 bits)


In [ ]:
subgroup_result = attack_largest_subgroup_with_pollard(weak_256_curve, max_iters=50_000_000, verbose=True)


## 80-bit demo

In [95]:
# Parameters from Sage for COMPOSITE-ORDER curve
p_c  = 847673381117647441179629
a_c  = 744006160578466613378795
b_c  = 300583906968757653399371
Gc_x = 98733059697549659874569
Gc_y = 834094303731412713283170
n_c  = 847673381117525559207585
factor_n_c = [(3, 1), (5, 1), (13, 1), (1747, 1), (3907, 1), (25841, 1), (44189, 1), (557743, 1)]

Gc = (Gc_x, Gc_y)

weak_80_curve = EllipticCurveGroup(
    p=p_c,
    a=a_c,
    b=b_c,
    G=Gc,
    n=n_c,
    factor_n=factor_n_c,
    name="Composite80Curve"
)

# sanity check
assert weak_80_curve.is_on_curve(Gc)
assert weak_80_curve.scalar_mul(n_c, Gc) is INF

# Print curve size and largest factor size
curve_bits = n_c.bit_length()   # or use p_c.bit_length()
largest_factor = max(q for (q, e) in factor_n_c)
largest_factor_bits = largest_factor.bit_length()

print(f"[{weak_80_curve.name}] Group order n is {n_c.bit_length()} bits")
print(f"[{weak_80_curve.name}] Largest prime factor = {largest_factor} ({largest_factor_bits} bits)")


[Composite80Curve] Group order n is 80 bits
[Composite80Curve] Largest prime factor = 557743 (20 bits)


In [162]:
k_secret_ph = weak_80_curve.random_scalar()
P_target_ph = weak_80_curve.scalar_mul(k_secret_ph)

k_ph, t_ph = time_call(
    weak_80_curve.pohlig_hellman,
    P_target_ph,
    verbose=True
)

print("PH - secret k:", k_secret_ph)
print("PH - found  k:", k_ph)
print("success:", (k_ph % n_c) == (k_secret_ph % n_c))
print("PH time:", t_ph, "s")

[Composite80Curve PH] starting Pohlig-Hellman
  factors: [(3, 1), (5, 1), (13, 1), (1747, 1), (3907, 1), (25841, 1), (44189, 1), (557743, 1)]
  [PH] solving modulo q=3 (e=1)
  [PH] solved mod 3 in 1 steps
    ⇒ congruence: k ≡ 0 (mod 3)
  [PH] solving modulo q=5 (e=1)
  [PH] solved mod 5 in 4 steps
    ⇒ congruence: k ≡ 3 (mod 5)
  [PH] solving modulo q=13 (e=1)
  [PH] solved mod 13 in 3 steps
    ⇒ congruence: k ≡ 2 (mod 13)
  [PH] solving modulo q=1747 (e=1)
  [PH] solved mod 1747 in 1567 steps
    ⇒ congruence: k ≡ 1566 (mod 1747)
  [PH] solving modulo q=3907 (e=1)
  [PH] solved mod 3907 in 1625 steps
    ⇒ congruence: k ≡ 1624 (mod 3907)
  [PH] solving modulo q=25841 (e=1)
  [PH] solved mod 25841 in 6186 steps
    ⇒ congruence: k ≡ 6185 (mod 25841)
  [PH] solving modulo q=44189 (e=1)
  [PH] solved mod 44189 in 30624 steps
    ⇒ congruence: k ≡ 30623 (mod 44189)
  [PH] solving modulo q=557743 (e=1)
  [PH] solved mod 557743 in 119181 steps
    ⇒ congruence: k ≡ 119180 (mod 557743)
  